**Demonstration of using simpy to simulate read receipts for
messages broadcast to 10 recipients.**

Messages are queued, sent, received and then read.

At each stage there is a random delay time.

Output is a json stream of {userid, timestamp, message status}

**Simpy idea**: Processes are represented by Python generators.

Docs: https://simpy.readthedocs.io/en/latest/
Simpy tutorial: https://www.youtube.com/watch?v=WHhJJxakIO4

In [1]:
import numpy as np
import json

# import simpy.rt # Uses 'real time' wall clock time. This won't work in a Jupyter notebook.
# env = simpy.rt.RealtimeEnvironment()

import simpy
env = simpy.Environment()

In [2]:
# Simulation parameters

NUM_RECIPIENTS = 10

SIM_DURATION = 1000

MIN_QUEUE_TIME = 1
MAX_QUEUE_TIME = 10

MIN_DELIVERY_TIME = 1
MAX_DELIVERY_TIME = 50

MIN_READ_TIME = 5
MAX_READ_TIME = 600

In [3]:
def get_outdict(time, userid, status):
    return {'time': time, 'userid': userid, 'Status': status}

def message_status(env, userid):
    outdict = get_outdict(env.now, userid, 'Queued')
    print(json.dumps(outdict))
    
    yield env.timeout(np.random.uniform(low=MIN_QUEUE_TIME,
                                        high=MAX_QUEUE_TIME))

    outdict = get_outdict(env.now, userid, 'Sent')
    print(json.dumps(outdict))
    
    yield env.timeout(np.random.uniform(low=MIN_DELIVERY_TIME,
                                        high=MAX_DELIVERY_TIME))

    outdict = get_outdict(env.now, userid, 'Delivered')
    print(json.dumps(outdict))
    
    yield env.timeout(np.random.uniform(low=MIN_READ_TIME,
                                        high=MAX_READ_TIME))

    outdict = get_outdict(env.now, userid, 'Read')
    print(json.dumps(outdict))

In [ ]:
# Setup and start the simulation

for userid in range(1, NUM_RECIPIENTS):
    message = message_status(env, userid)
    env.process(message)

env.run(until=SIM_DURATION)